<a href="https://colab.research.google.com/github/NeuralClassifier/OutlierDetect/blob/main/HDBSCAN_optimalHierarchies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
!unzip /content/HDBSCAN_outlier_noise.zip

In [ ]:
!unzip /content/HDBSCAN_outlier.zip

In [ ]:
!unzip /content/reduced_synthetic_data_local_global.zip

In [62]:
hdb_backbone_path = '/content/HDBSCAN_outlier/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

def perf_measure(y_actual, y_hat):
  TP = 0
  FP = 0
  TN = 0
  FN = 0

  for i in range(len(y_hat)):

    if y_actual[i]==y_hat[i]==1:
      TP += 1
    if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
      FP += 1
    if y_actual[i]==y_hat[i]==0:
      TN += 1
    if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
      FN += 1

  return TP, FP, TN, FN


def LOF_tp_fp_fn_tn(d1,d2,r,o,d):
  
  mpts_list = list(d1['mpts'].unique())
  mpts_tr = []
  reduc = []
  ovr = []
  dm = []
  tp = []
  fp = []
  tn = []
  fn = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  y_actual = np.array(d2.iloc[:,-1])
  for mpts in mpts_list:
    #clear_output(wait=True)
    d1_temp = d1[d1['mpts']==mpts]
    #mpts_list=list(d1_temp['mpts'].unique())
    mpts_tr.append(mpts)
    reduc.append(r)
    ovr.append(o)
    dm.append(d)
    data = []
    #data_idx = list(d1_temp['data index'].unique())
    #print(d1_temp)
    y_hat = np.array(d1_temp.iloc[:,-1])
    TP, FP, TN, FN =  perf_measure(y_actual,y_hat)
    tp.append(TP)
    fp.append(FP)
    tn.append(TN)
    fn.append(FN)
    print('TP: ',TP,' FP: ',FP,' TN: ',TN,' FN: ',FN)
  reduc = pd.DataFrame(reduc,columns=['Outlier Quantity'])
  ovr = pd.DataFrame(ovr,columns=['Overlap'])
  dm = pd.DataFrame(dm,columns=['Dimension'])
  mpts_tr = pd.DataFrame(mpts_tr,columns=['mpts'])
  tp = pd.DataFrame(tp,columns=['TP'])
  fp = pd.DataFrame(fp,columns=['FP'])
  tn = pd.DataFrame(tn,columns=['TN'])
  fn = pd.DataFrame(fn,columns=['FN'])
  
  precAtn, adjusted_precAtn, n_out, n_in = prec_at_n_lof(d1,d2)

  return pd.concat([reduc,ovr,dm,mpts_tr,n_out, n_in,tp,fp,tn,fn,precAtn,adjusted_precAtn],axis=1)



def KNN_tp_fp_fn_tn(d1,d2,r,o,d):
  
  mpts_list = list(d1['mpts'].unique())
  mpts_tr = []
  reduc = []
  ovr = []
  dm = []
  tp = []
  fp = []
  tn = []
  fn = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  y_actual = np.array(d2.iloc[:,-2])
  for mpts in mpts_list:
    #clear_output(wait=True)
    d1_temp = d1[d1['mpts']==mpts]
    #mpts_list=list(d1_temp['mpts'].unique())
    mpts_tr.append(mpts)
    reduc.append(r)
    ovr.append(o)
    dm.append(d)
    data = []
    #data_idx = list(d1_temp['data index'].unique())
    #print(d1_temp)
    y_hat = np.array(d1_temp.iloc[:,-1])
    TP, FP, TN, FN =  perf_measure(y_actual,y_hat)
    tp.append(TP)
    fp.append(FP)
    tn.append(TN)
    fn.append(FN)
    print('TP: ',TP,' FP: ',FP,' TN: ',TN,' FN: ',FN)
  reduc = pd.DataFrame(reduc,columns=['Outlier Quantity'])
  ovr = pd.DataFrame(ovr,columns=['Overlap'])
  dm = pd.DataFrame(dm,columns=['Dimension'])
  mpts_tr = pd.DataFrame(mpts_tr,columns=['mpts'])
  tp = pd.DataFrame(tp,columns=['TP'])
  fp = pd.DataFrame(fp,columns=['FP'])
  tn = pd.DataFrame(tn,columns=['TN'])
  fn = pd.DataFrame(fn,columns=['FN'])
  
  precAtn, adjusted_precAtn, n_out, n_in = prec_at_n_KNN(d1,d2)

  return pd.concat([reduc,ovr,dm,mpts_tr,n_out, n_in,tp,fp,tn,fn,precAtn,adjusted_precAtn],axis=1)




def prec_at_n_KNN(d1,d2):
  out_idx = []
  mpts_tr = []
  prec_at_n = []
  ap_lst = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  for i in range(len(d2)):
    if d2.iloc[i,-2] == 1:
      out_idx.append(d2.iloc[i,0])
  #print(out_idx)

  o = 0
  i = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
    else:
      i += 1

  n_out = []
  n_in = []
  mpts_list = list(d1['mpts'].unique())
  for mpts in mpts_list:
    mpts_tr.append(mpts)
    num_cnt =0
    d1_temp = d1[d1['mpts']==mpts]
    for i in range(len(d1_temp)):
      if d1_temp.iloc[i,-1] == 1:
        if d1_temp.iloc[i,1] in out_idx:
          num_cnt+=1
    p_at_n = float(num_cnt/len(out_idx))
    if o>n:
      npm = 1
    else:
      npm = float(o/n)
    ap = float((p_at_n-npm)/(1-npm))
    prec_at_n.append(p_at_n)
    ap_lst.append(ap)
    n_out.append(o)
    n_in.append(i)
  #print(d3)
  prec_at_n = pd.DataFrame(prec_at_n,columns=['P@n'])
  ap_lst = pd.DataFrame(ap_lst,columns=['Adjusted P@n'])
  n_out = pd.DataFrame(n_out,columns=['N_out'])
  n_in = pd.DataFrame(n_in,columns=['N_in'])
  #print(prec_at_n)
  #df_res = pd.concat([d3,prec_at_n],axis = 1)
  return prec_at_n, ap_lst, n_out, n_in




# n is the total number of outliers
def prec_at_n_lof(d1,d2):
  out_idx = []
  mpts_tr = []
  prec_at_n = []
  ap_lst = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      out_idx.append(d2.iloc[i,0])
  #print(out_idx)

  o = 0
  i = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
    else:
      i += 1

  n_out = []
  n_in = []
  mpts_list = list(d1['mpts'].unique())
  for mpts in mpts_list:
    mpts_tr.append(mpts)
    num_cnt =0
    d1_temp = d1[d1['mpts']==mpts]
    for i in range(len(d1_temp)):
      if d1_temp.iloc[i,-1] == 1:
        if d1_temp.iloc[i,1] in out_idx:
          num_cnt+=1
    p_at_n = float(num_cnt/len(out_idx))
    if o>n:
      npm = 1
    else:
      npm = float(o/n)
    ap = float((p_at_n-npm)/(1-npm))
    prec_at_n.append(p_at_n)
    ap_lst.append(ap)
    n_out.append(o)
    n_in.append(i)
  #print(d3)
  prec_at_n = pd.DataFrame(prec_at_n,columns=['P@n'])
  ap_lst = pd.DataFrame(ap_lst,columns=['Adjusted P@n'])
  n_out = pd.DataFrame(n_out,columns=['N_out'])
  n_in = pd.DataFrame(n_in,columns=['N_in'])
  #print(prec_at_n)
  #df_res = pd.concat([d3,prec_at_n],axis = 1)
  return prec_at_n, ap_lst, n_out, n_in

def ap_lof(d2,d3):
  o = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
  ap_lst = []
  mpts_lst = list(d3['mpts'].unique())
  for i in mpts_lst:
    ap_lst = []
    d3_temp = d3[d3['mpts']==i] 
    for j in range(len(d3_temp)):
      p_at_n = d3_temp.iloc[j,-1]
      ap = float((p_at_n-(o/n))/(1-(o/n)))
      ap_lst.append(ap)
    

In [63]:
#LOF

hdb_backbone_path = '/content/HDBSCAN_outlier/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

for r in redc:
  redc_path = hdb_backbone_path + str(r)+'p/'
  for o in over:
    over_path = redc_path + str(o)+'/'
    for d in dim:
      clear_output(wait=True)
      final_path_hdb = over_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'_hdbLabls.xlsx'
      print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d)
      df_hdb = pd.read_excel(final_path_hdb)
      df_hdb.drop(['Unnamed: 0'],axis=1,inplace=True)

      path_real = '/content/reduced_synthetic_data_local_global/'+str(r)+'p/'+str(o)+'/combined_global_local/'+str(r)+'p_m'+str(o)+'_dim'+str(d)+'_combined.xlsx'
      df_real = pd.read_excel(path_real)
      df_real.drop(['Unnamed: 0'],axis=1,inplace=True)
      if r == 30 and o == 1 and d == 2:
        df_res = LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d)
      else:
        df_res = pd.concat([df_res,LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d)],axis=0)

Reduction:  90 , Overlap:  4 , Dimension:  100
TP:  11  FP:  7  TN:  188  FN:  14
TP:  9  FP:  9  TN:  186  FN:  16
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  5  FP:  13  TN:  182  FN:  20
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  TN:  195  FN:  7
TP:  18  FP:  0  

In [64]:
df_res.reset_index(drop=True, inplace=True)
df_res.to_excel('/content/performance_rodm_LOF.xlsx')
df_res

,Outlier Quantity,Overlap,Dimension,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,30,1,2,2,7,339,1,27,306,6,0.142857,0.124839
1,30,1,2,3,7,339,5,23,310,2,0.714286,0.708280
2,30,1,2,4,7,339,6,22,311,1,0.857143,0.854140
3,30,1,2,5,7,339,7,21,312,0,1.000000,1.000000
4,30,1,2,6,7,339,7,21,312,0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
26683,90,4,100,215,25,219,3,2,193,22,0.120000,0.007179
26684,90,4,100,216,25,219,3,1,194,22,0.120000,0.007179
26685,90,4,100,217,25,219,2,1,194,23,0.080000,-0.037949
26686,90,4,100,218,25,219,1,1,194,24,0.040000,-0.083077


In [65]:
#KNN Outlier

hdb_backbone_path = '/content/HDBSCAN_outlier/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

for r in redc:
  redc_path = hdb_backbone_path + str(r)+'p/'
  for o in over:
    over_path = redc_path + str(o)+'/'
    for d in dim:
      clear_output(wait=True)
      final_path_hdb = over_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'_hdbLabls.xlsx'
      print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d)
      df_hdb = pd.read_excel(final_path_hdb)
      df_hdb.drop(['Unnamed: 0'],axis=1,inplace=True)

      path_real = '/content/reduced_synthetic_data_local_global/'+str(r)+'p/'+str(o)+'/combined_global_local/'+str(r)+'p_m'+str(o)+'_dim'+str(d)+'_combined.xlsx'
      df_real = pd.read_excel(path_real)
      df_real.drop(['Unnamed: 0'],axis=1,inplace=True)
      if r == 30 and o == 1 and d == 2:
        df_res = KNN_tp_fp_fn_tn(df_hdb,df_real,r,o,d)
      else:
        df_res = pd.concat([df_res,KNN_tp_fp_fn_tn(df_hdb,df_real,r,o,d)],axis=0)

Reduction:  90 , Overlap:  4 , Dimension:  100
TP:  16  FP:  2  TN:  176  FN:  26
TP:  18  FP:  0  TN:  178  FN:  24
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  17  FP:  1  TN:  177  FN:  25
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  16  FP:  2  TN:  176  FN:  26
TP:  14  FP:  4  TN:  174  FN:  28
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15  FP:  3  TN:  175  FN:  27
TP:  15 

In [66]:
df_res.reset_index(drop=True, inplace=True)
df_res.to_excel('/content/performance_rodm_KNN.xlsx')
df_res

,Outlier Quantity,Overlap,Dimension,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,30,1,2,2,7,339,2,26,265,47,0.040816,0.020653
1,30,1,2,3,7,339,7,21,270,42,0.142857,0.124839
2,30,1,2,4,7,339,15,13,278,34,0.306122,0.291536
3,30,1,2,5,7,339,20,8,283,29,0.408163,0.395722
4,30,1,2,6,7,339,28,0,291,21,0.571429,0.562420
...,...,...,...,...,...,...,...,...,...,...,...,...
26683,90,4,100,215,25,219,4,1,177,38,0.095238,-0.020757
26684,90,4,100,216,25,219,4,0,178,38,0.095238,-0.020757
26685,90,4,100,217,25,219,3,0,178,39,0.071429,-0.047619
26686,90,4,100,218,25,219,2,0,178,40,0.047619,-0.074481


In [ ]:
df_hdb

,mpts,data index,outlier score,HDBSCAN_outliers
0,2,0,0.000000,0
1,2,1,0.240688,0
2,2,2,0.000000,0
3,2,3,0.031608,0
4,2,4,0.275930,0
...,...,...,...,...
114915,339,335,0.000000,0
114916,339,336,0.000000,0
114917,339,337,0.000000,0
114918,339,338,0.000000,0


In [ ]:
df_real.sort_values(by=['data index'], ascending=True,inplace=True)
df_real

,data index,A1,A2,Y,normalized_LOF,knn_outlier,LOF_outlier
23,0,2.764052,1.400157,0,0.427347,1,0
15,1,1.978738,3.240893,0,0.492060,1,0
24,2,2.867558,0.022722,0,0.425059,1,0
303,3,1.950088,0.848643,0,0.032013,0,0
278,4,0.896781,1.410599,0,0.038716,0,0
...,...,...,...,...,...,...,...
83,335,-1.259781,0.058599,1,0.179400,0,0
116,336,-0.657871,0.049916,1,0.122259,0,0
155,337,-0.797401,1.412470,1,0.086318,0,0
122,338,0.174586,-0.977099,1,0.111748,0,0


In [ ]:
prec_at_n_lof(df_hdb,df_real,df_res)

     Outlier Quantity  Overlap  Dimension  mpts  TP  FP   TN  FN
0                  30        1          2     2   1  27  306   6
1                  30        1          2     3   5  23  310   2
2                  30        1          2     4   6  22  311   1
3                  30        1          2     5   7  21  312   0
4                  30        1          2     6   7  21  312   0
..                ...      ...        ...   ...  ..  ..  ...  ..
213                90        4        100   215   3   2  193  22
214                90        4        100   216   3   1  194  22
215                90        4        100   217   2   1  194  23
216                90        4        100   218   1   1  194  24
217                90        4        100   219   0   1  194  25

[26688 rows x 8 columns]
     PrecAtN
0       0.44
1       0.36
2       0.20
3       0.20
4       0.20
..       ...
213     0.12
214     0.12
215     0.08
216     0.04
217     0.00

[218 rows x 1 columns]


ValueError: ignored

In [78]:
##run this for noise

hdb_backbone_path = '/content/HDBSCAN_outlier_noise/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

def perf_measure(y_actual, y_hat):
  TP = 0
  FP = 0
  TN = 0
  FN = 0

  for i in range(len(y_hat)):

    if y_actual[i]==y_hat[i]==1:
      TP += 1
    if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
      FP += 1
    if y_actual[i]==y_hat[i]==0:
      TN += 1
    if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
      FN += 1

  return TP, FP, TN, FN


def LOF_tp_fp_fn_tn(d1,d2,r,o,d,n):
  
  mpts_list = list(d1['mpts'].unique())
  mpts_tr = []
  reduc = []
  ovr = []
  dm = []
  no = []
  tp = []
  fp = []
  tn = []
  fn = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  y_actual = np.array(d2.iloc[:,-1])
  for mpts in mpts_list:
    #clear_output(wait=True)
    d1_temp = d1[d1['mpts']==mpts]
    #mpts_list=list(d1_temp['mpts'].unique())
    mpts_tr.append(mpts)
    reduc.append(r)
    ovr.append(o)
    dm.append(d)
    no.append(n)
    data = []
    #data_idx = list(d1_temp['data index'].unique())
    #print(d1_temp)
    y_hat = np.array(d1_temp.iloc[:,-1])
    TP, FP, TN, FN =  perf_measure(y_actual,y_hat)
    tp.append(TP)
    fp.append(FP)
    tn.append(TN)
    fn.append(FN)
    print('TP: ',TP,' FP: ',FP,' TN: ',TN,' FN: ',FN)
  reduc = pd.DataFrame(reduc,columns=['Outlier Quantity'])
  ovr = pd.DataFrame(ovr,columns=['Overlap'])
  dm = pd.DataFrame(dm,columns=['Dimension'])
  no = pd.DataFrame(no,columns=['Noise'])
  mpts_tr = pd.DataFrame(mpts_tr,columns=['mpts'])
  tp = pd.DataFrame(tp,columns=['TP'])
  fp = pd.DataFrame(fp,columns=['FP'])
  tn = pd.DataFrame(tn,columns=['TN'])
  fn = pd.DataFrame(fn,columns=['FN'])
  
  precAtn, adjusted_precAtn, n_out, n_in = prec_at_n_lof(d1,d2)

  return pd.concat([reduc,ovr,dm,no,mpts_tr,n_out, n_in,tp,fp,tn,fn,precAtn,adjusted_precAtn],axis=1)



def KNN_tp_fp_fn_tn(d1,d2,r,o,d,n):
  
  mpts_list = list(d1['mpts'].unique())
  mpts_tr = []
  reduc = []
  ovr = []
  dm = []
  no = []
  tp = []
  fp = []
  tn = []
  fn = []
  print(d2)
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  y_actual = np.array(d2.iloc[:,-2])
  for mpts in mpts_list:
    #clear_output(wait=True)
    d1_temp = d1[d1['mpts']==mpts]
    #mpts_list=list(d1_temp['mpts'].unique())
    mpts_tr.append(mpts)
    reduc.append(r)
    ovr.append(o)
    dm.append(d)
    no.append(n)
    data = []
    #data_idx = list(d1_temp['data index'].unique())
    #print(d1_temp)
    y_hat = np.array(d1_temp.iloc[:,-1])
    TP, FP, TN, FN =  perf_measure(y_actual,y_hat)
    tp.append(TP)
    fp.append(FP)
    tn.append(TN)
    fn.append(FN)
    print('TP: ',TP,' FP: ',FP,' TN: ',TN,' FN: ',FN)
  reduc = pd.DataFrame(reduc,columns=['Outlier Quantity'])
  ovr = pd.DataFrame(ovr,columns=['Overlap'])
  dm = pd.DataFrame(dm,columns=['Dimension'])
  no = pd.DataFrame(no,columns=['Noise'])
  mpts_tr = pd.DataFrame(mpts_tr,columns=['mpts'])
  tp = pd.DataFrame(tp,columns=['TP'])
  fp = pd.DataFrame(fp,columns=['FP'])
  tn = pd.DataFrame(tn,columns=['TN'])
  fn = pd.DataFrame(fn,columns=['FN'])
  
  precAtn, adjusted_precAtn, n_out, n_in = prec_at_n_knn(d1,d2)

  return pd.concat([reduc,ovr,dm,no,mpts_tr,n_out, n_in,tp,fp,tn,fn,precAtn,adjusted_precAtn],axis=1)




# n is the total number of outliers
def prec_at_n_lof(d1,d2):
  out_idx = []
  mpts_tr = []
  prec_at_n = []
  ap_lst = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      out_idx.append(d2.iloc[i,0])
  #print(out_idx)

  o = 0
  i = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
    else:
      i += 1

  n_out = []
  n_in = []
  mpts_list = list(d1['mpts'].unique())
  print(d1)
  for mpts in mpts_list:
    mpts_tr.append(mpts)
    num_cnt =0
    d1_temp = d1[d1['mpts']==mpts]
    ol_pred = []
    for i in range(len(d1_temp)):
      if d1_temp.iloc[i,-1] == 1:
        if d1_temp.iloc[i,0] in out_idx:
          ol_pred.append(d1_temp.iloc[i,1])
          num_cnt+=1
    p_at_n = float(num_cnt/len(out_idx))
    if p_at_n > 1:
      print('>1 mpts = ',mpts)
      print('>1 num_cnt = ',num_cnt)
      print('>1 len(out_idx) = ',len(out_idx))
      print('>1 len(df1_temp) = ',len(d1_temp))
      print('>1 pred out idx',ol_pred)
      print('>1 act out idx',out_idx)
      print(d1_temp[d1_temp['mpts']==mpts])
    if o>n:
      npm = 1
    else:
      npm = float(o/n)
    ap = float((p_at_n-npm)/(1-npm))
    prec_at_n.append(p_at_n)
    ap_lst.append(ap)
    n_out.append(o)
    n_in.append(i)
  #print(d3)
  prec_at_n = pd.DataFrame(prec_at_n,columns=['P@n'])
  ap_lst = pd.DataFrame(ap_lst,columns=['Adjusted P@n'])
  n_out = pd.DataFrame(n_out,columns=['N_out'])
  n_in = pd.DataFrame(n_in,columns=['N_in'])
  #print(prec_at_n)
  #df_res = pd.concat([d3,prec_at_n],axis = 1)
  return prec_at_n, ap_lst, n_out, n_in

# n is the total number of outliers
def prec_at_n_knn(d1,d2):
  out_idx = []
  mpts_tr = []
  prec_at_n = []
  ap_lst = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  for i in range(len(d2)):
    if d2.iloc[i,-2] == 1:
      out_idx.append(d2.iloc[i,0])
  #print(out_idx)

  o = 0
  i = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
    else:
      i += 1

  n_out = []
  n_in = []
  mpts_list = list(d1['mpts'].unique())
  print(d1)
  for mpts in mpts_list:
    mpts_tr.append(mpts)
    num_cnt =0
    d1_temp = d1[d1['mpts']==mpts]
    ol_pred = []
    for i in range(len(d1_temp)):
      if d1_temp.iloc[i,-1] == 1:
        if d1_temp.iloc[i,0] in out_idx:
          ol_pred.append(d1_temp.iloc[i,1])
          num_cnt+=1
    p_at_n = float(num_cnt/len(out_idx))
    if p_at_n > 1:
      print('>1 mpts = ',mpts)
      print('>1 num_cnt = ',num_cnt)
      print('>1 len(out_idx) = ',len(out_idx))
      print('>1 len(df1_temp) = ',len(d1_temp))
      print('>1 pred out idx',ol_pred)
      print('>1 act out idx',out_idx)
      print(d1_temp[d1_temp['mpts']==mpts])
    if o>n:
      npm = 1
    else:
      npm = float(o/n)
    ap = float((p_at_n-npm)/(1-npm))
    prec_at_n.append(p_at_n)
    ap_lst.append(ap)
    n_out.append(o)
    n_in.append(i)
  #print(d3)
  prec_at_n = pd.DataFrame(prec_at_n,columns=['P@n'])
  ap_lst = pd.DataFrame(ap_lst,columns=['Adjusted P@n'])
  n_out = pd.DataFrame(n_out,columns=['N_out'])
  n_in = pd.DataFrame(n_in,columns=['N_in'])
  #print(prec_at_n)
  #df_res = pd.concat([d3,prec_at_n],axis = 1)
  return prec_at_n, ap_lst, n_out, n_in

In [79]:
#LOF Noise

hdb_backbone_path = '/content/HDBSCAN_outlier_noise/'
redc = [70]
over = [4]
dim = [2,10,20,40,80,100]
noise = [20,30,40,50]
for r in redc:
  for o in over:
    for d in dim:
      for n in noise:
        #clear_output(wait=True)
        final_path_hdb = hdb_backbone_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'n'+str(n)+'_hdbLabls.xlsx'
        print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d,', Noise: ',n)
        df_hdb = pd.read_excel(final_path_hdb)
        df_hdb.drop(['Unnamed: 0'],axis=1,inplace=True)

        path_real = '/content/reduced_synthetic_data_local_global/'+str(r)+'p/'+str(o)+'/combined_global_local/'+str(r)+'p_m'+str(o)+'_dim'+str(d)+'_combined.xlsx'
        df_real = pd.read_excel(path_real)
        df_real.drop(['Unnamed: 0'],axis=1,inplace=True)
        if r == 70 and o == 4 and d == 2 and n == 20:
          df_res = LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d,n)
        else:
          df_res = pd.concat([df_res,LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d,n)],axis=0)

Streaming output truncated to the last 5000 lines.
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN:  4
TP:  1  FP:  20  TN:  235  FN: 

In [80]:
df_res

,Outlier Quantity,Overlap,Dimension,Noise,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,70,4,2,20,2,7,259,1,20,233,6,0.142857,0.119142
1,70,4,2,20,3,7,259,0,21,232,7,0.000000,-0.027668
2,70,4,2,20,4,7,259,2,19,234,5,0.285714,0.265951
3,70,4,2,20,5,7,259,4,17,236,3,0.571429,0.559571
4,70,4,2,20,6,7,259,4,17,236,3,0.571429,0.559571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,70,4,100,50,255,8,259,0,21,231,8,0.000000,-0.031746
254,70,4,100,50,256,8,259,0,21,231,8,0.000000,-0.031746
255,70,4,100,50,257,8,259,0,21,231,8,0.000000,-0.031746
256,70,4,100,50,258,8,259,0,21,231,8,0.000000,-0.031746


In [81]:
df_res.reset_index(drop=True, inplace=True)
df_res.to_excel('/content/performance_dn_LOF_noise.xlsx')
df_res

,Outlier Quantity,Overlap,Dimension,Noise,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,70,4,2,20,2,7,259,1,20,233,6,0.142857,0.119142
1,70,4,2,20,3,7,259,0,21,232,7,0.000000,-0.027668
2,70,4,2,20,4,7,259,2,19,234,5,0.285714,0.265951
3,70,4,2,20,5,7,259,4,17,236,3,0.571429,0.559571
4,70,4,2,20,6,7,259,4,17,236,3,0.571429,0.559571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6187,70,4,100,50,255,8,259,0,21,231,8,0.000000,-0.031746
6188,70,4,100,50,256,8,259,0,21,231,8,0.000000,-0.031746
6189,70,4,100,50,257,8,259,0,21,231,8,0.000000,-0.031746
6190,70,4,100,50,258,8,259,0,21,231,8,0.000000,-0.031746


In [86]:
gg= df_res[df_res['Noise']==30]
list(gg['Dimension'].unique())

[2, 10, 20, 40, 80, 100]

In [83]:
#KNN Noise

hdb_backbone_path = '/content/HDBSCAN_outlier_noise/'
redc = [70]
over = [4]
dim = [2,10,20,40,80,100]
noise = [20,30,40,50]
for r in redc:
  for o in over:
    for d in dim:
      for n in noise:
        clear_output(wait=True)
        final_path_hdb = hdb_backbone_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'n'+str(n)+'_hdbLabls.xlsx'
        print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d,', Noise: ',n)
        df_hdb = pd.read_excel(final_path_hdb)
        df_hdb.drop(['Unnamed: 0'],axis=1,inplace=True)

        path_real = '/content/reduced_synthetic_data_local_global/'+str(r)+'p/'+str(o)+'/combined_global_local/'+str(r)+'p_m'+str(o)+'_dim'+str(d)+'_combined.xlsx'
        df_real = pd.read_excel(path_real)
        df_real.drop(['Unnamed: 0'],axis=1,inplace=True)
        if r == 70 and o == 4 and d == 2 and n == 20:
          df_res = KNN_tp_fp_fn_tn(df_hdb,df_real,r,o,d,n)
        else:
          df_res = pd.concat([df_res,KNN_tp_fp_fn_tn(df_hdb,df_real,r,o,d,n)],axis=0)

Streaming output truncated to the last 5000 lines.
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP:  4  FP:  17  TN:  217  FN:  22
TP: 

In [84]:
df_res

,Outlier Quantity,Overlap,Dimension,Noise,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,70,4,2,20,2,7,259,3,18,218,21,0.125000,0.100791
1,70,4,2,20,3,7,259,8,13,223,16,0.333333,0.314888
2,70,4,2,20,4,7,259,19,2,234,5,0.791667,0.785903
3,70,4,2,20,5,7,259,17,4,232,7,0.708333,0.700264
4,70,4,2,20,6,7,259,17,4,232,7,0.708333,0.700264
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,70,4,100,50,255,8,259,12,9,208,31,0.279070,0.256183
254,70,4,100,50,256,8,259,12,9,208,31,0.279070,0.256183
255,70,4,100,50,257,8,259,12,9,208,31,0.279070,0.256183
256,70,4,100,50,258,8,259,12,9,208,31,0.279070,0.256183


In [85]:
df_res.reset_index(drop=True, inplace=True)
df_res.to_excel('/content/performance_dn_KNN_noise.xlsx')
df_res

,Outlier Quantity,Overlap,Dimension,Noise,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,70,4,2,20,2,7,259,3,18,218,21,0.125000,0.100791
1,70,4,2,20,3,7,259,8,13,223,16,0.333333,0.314888
2,70,4,2,20,4,7,259,19,2,234,5,0.791667,0.785903
3,70,4,2,20,5,7,259,17,4,232,7,0.708333,0.700264
4,70,4,2,20,6,7,259,17,4,232,7,0.708333,0.700264
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6187,70,4,100,50,255,8,259,12,9,208,31,0.279070,0.256183
6188,70,4,100,50,256,8,259,12,9,208,31,0.279070,0.256183
6189,70,4,100,50,257,8,259,12,9,208,31,0.279070,0.256183
6190,70,4,100,50,258,8,259,12,9,208,31,0.279070,0.256183
